In [ ]:
pip install requests beautifulsoup4

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
url = 'https://www.basketball-reference.com/teams/PHI/2023_games.html'
response = requests.get(url)

In [ ]:
soup = BeautifulSoup(response.content, 'html.parser')
table = soup.find('table', {'id': 'games'})

In [4]:
box_score_url =[]

for row in table.find_all('tr'):
    regular_season_section = soup.find('div', {'id': 'all_games'})
    regular_season_table = regular_season_section.find('table', {'id': 'games'})
    for row in regular_season_table.find_all('tr'):
        box_score_link = row.find('a', text='Box Score')
        if box_score_link:
            box_score_url.append(box_score_link['href'])


In [ ]:
len(box_score_url)
box_score_url

In [7]:
data = []
sleep = 5

for url in box_score_url[7:10]:
    box_score_response = requests.get("https://www.basketball-reference.com" + url)
    print (box_score_response)
    box_score_content = box_score_response.content
    box_score_soup = BeautifulSoup(box_score_content, 'html.parser')

    stats_match = box_score_soup.find(id='div_box-PHI-game-basic')
    stats_match_table = stats_match.find('table')

    stats_h1 = box_score_soup.find(id='all_box-PHI-h1-basic')
    stats_h1_table = stats_h1.find('table')

    rows_to_collect = [18]
    rows_to_collect_labels = [2]
    collected_rows_match = []
    collected_rows_h1 = []
    collected_rows_labels = []
    row_data_match = []
    row_data_h1 = []
    row_data_h11 = []
    result_cell = []
    labels = []

    for i, row in enumerate(stats_match_table.find_all('tr')):
        if i + 1 in rows_to_collect_labels:
            collected_rows_labels.append(row)

    for row in collected_rows_labels:
        cells = row.find_all('th')
        labels = [cell.get_text(strip=True) for cell in cells]

    for i, row in enumerate(stats_match_table.find_all('tr')):
        if i + 1 in rows_to_collect:
            collected_rows_match.append(row)

    for row in collected_rows_match:
        cells = row.find_all('td')
        result_cell.append((cells[18].get_text(strip=True)))


    for i, row in enumerate(stats_h1_table.find_all('tr')):
        if i + 1 in rows_to_collect:
            collected_rows_h1.append(row)

    for row in collected_rows_h1:
        cells = row.find_all('td')
        starters = row.find_all('th')
        row_data_h1 = [starter.get_text(strip=True) for starter in starters[:-1]] + [cell.get_text(strip=True) for cell in cells[:-1]] + result_cell

    data.append(row_data_h1)
    time.sleep(sleep)
    sleep = sleep + 5

<Response [200]>
<Response [200]>
<Response [200]>


In [ ]:
df = pd.DataFrame(data)
df.columns = labels [1:21]
new_column_name = 'Target'
df = df.rename(columns={df.columns[-1]: new_column_name})


In [244]:
df

,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Target
0,120,20,40,.500,9,20,.450,14,17,.824,4,11,15,8,4,3,7,13,63,117
1,120,19,43,.442,4,11,.364,3,5,.600,5,19,24,9,3,1,5,5,45,88
2,120,16,43,.372,6,17,.353,13,17,.765,5,19,24,10,3,4,6,6,51,105
3,120,21,46,.457,9,26,.346,13,14,.929,6,19,25,15,6,4,3,11,64,120
4,120,19,39,.487,6,13,.462,9,13,.692,2,18,20,11,0,1,6,5,53,109
5,120,25,44,.568,12,23,.522,3,3,1.000,4,12,16,16,5,2,4,10,65,112
6,120,24,45,.533,10,20,.500,6,7,.857,5,15,20,17,4,3,6,12,64,114
7,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
8,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
9,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [246]:
df = df.drop(df.tail(3).index)

file_path = 'nba_predictions.csv'

df.to_csv(file_path, index=False)

print("Data saved to CSV successfully.")


Data saved to CSV successfully.


In [ ]:
file_path = 'nba_predictions.csv'

# Append the new data to the CSV file
new_data.to_csv(file_path, mode='a', header=False, index=False)

print("New data appended to CSV successfully.")